# Libraries

In [1]:
%load_ext rpy2.ipython


In [2]:
%%R
library(tidyverse)
library(ggtree)
library(ggstance)
library(Cairo)
library(showtext)
library(stringr)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidytree
  groupOTU.treedata   tidytree
  is.rooted.treedata  tidytree
  nodeid.phylo        tidytree
  nodeid.treedata     tidytree
  nodelab.phylo       tidytree
  nodelab.treedata    tidytree
  offspring.phylo     tidytree
  offspring.treedata  tidytree
  parent.phylo        tidytree
  parent.treedata     tidytree
  root.treedata       tidytree
  rootnode.phylo      tidytree
  sibling.phylo       tidytree
ggtree v3.6.2 For help: https://yulab-smu.top/treedata-book/

If you use the ggtree package suite in published resea

In [3]:
%%R
genome_data <- read_csv("data/Enterobacteriaciae_species_merged_1000.csv")

# head(genome_data)


Rows: 8287 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): accession, ncbi_organism_name, ncbi_taxonomy, gtdb_taxonomy
lgl (2): gtdb_species_representative, ncbi_type_material

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
%%R
genomes_downloaded <- read_csv("data/downloaded_genomes_list.txt", col_names = FALSE)
length(genomes_downloaded)

# head(genomes_downloaded)


Rows: 7981 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] 1


In [5]:
%%R
#select genomes in the genomes_downloaded list
genome_data <- genome_data[genome_data$accession %in% genomes_downloaded$X1,]
length(genome_data$accession)


[1] 7981


In [6]:
# %%R
# hmm_hits <- hmm_hits %>%
# 	separate(genome, c('db','bacteria','genome','contig'), sep = '/')
# head(hmm_hits$genome)

In [7]:
# %%R
# head(genome_data$accession)

In [8]:
# %%R
# #save hmm_hits
# write.csv(hmm_hits, "data/20230608_40fullmic_hmm_hits.csv", row.names = FALSE)

In [9]:
# %%R
# hmm_hits <- read.csv("data/20230622_cinful10_best_hits.csv")

# # head(hmm_hits)


In [10]:
# %%R

# # Make 'genome' column from 'sample' column, remove 'refseq/bacteria/' from the beginning of the string and after the next '/'
# hmm_hits$genome <- gsub('refseq/bacteria/', '', hmm_hits$sample)
# hmm_hits$genome <- gsub('/.*', '', hmm_hits$genome)

# # head(hmm_hits, 6)


In [11]:
# %%R

# # Make new blank column in hmm_hits dataframe
# hmm_hits$gtdb_taxonomy <- NA

# # count rows from genome_data with no contig information
# genome_data[is.na(genome_data$gtdb_taxonomy),]


# # Match the first 13 characters of the contig column between hmm_hits and all_best_hits
# # If there is a match, copy the gtdb_taxonomy from genome_data to hmm_hits
# for (i in 1:nrow(hmm_hits)){
#     for (j in 1:nrow(genome_data)){
#         if (substr(hmm_hits$genome[i], 1, 13) == substr(genome_data$accession[j], 1, 13)){
#             hmm_hits$gtdb_taxonomy[i] <- genome_data$gtdb_taxonomy[j]
#         }
#     }
# }


# # Print the updated dataframe
# head(hmm_hits$gtdb_taxonomy)

# save hmm_hits
# write.csv(hmm_hits, "data/20230622_cinful10_best_hits_35minute_timesave.csv", row.names = FALSE)


In [12]:
%%R
#read in the saved file (for a timesave)
hmm_hits <- read_csv("data/20230622_cinful10_best_hits_35minute_timesave.csv")

#head(hmm_hits$component)
#head(hmm_hits$hmmerHit)

Rows: 34337 Columns: 30
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): qseqid, sseqid, component, cinful_id, pephash, sample, contig, sta...
dbl (14): pident, length, mismatch, gapopen, qstart, qend, sstart, send, eva...
lgl  (4): hmmerHit, verified, allStandardAA, signalMatch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [13]:
%%R
# keep hmm_hits where $component == 'microcins.verified' and $hmmerHit == 'True'
hmm_hits <- hmm_hits[hmm_hits$component == 'microcins.verified' & hmm_hits$hmmerHit == 'TRUE',]
# head(hmm_hits)

# print number of rows
nrow(hmm_hits)


[1] 6427


In [14]:
%%R
Enterobacteriaciae_species_tree <- read.tree("data/Enterobacteriaciae_species.tree")


In [15]:
%%R
samples <- read.csv("data/Enterobacteriaciae_accessions.csv", header = TRUE)
samples <- samples[samples$accession %in% genomes_downloaded$X1,]
length(samples$accession)
#head(samples)

[1] 7981


In [16]:
%%R
samples %>%
	separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
	count(f8) %>%
	#remove NA
	na.omit()

                                   f8   n
1                Actinomyces marmotae   5
2           Anaerotruncus colihominis   1
3             Atlantibacter hermannii  28
4              Brachyspira pilosicoli   1
5                Campylobacter_D coli  33
6                 Citrobacter braakii 119
7               Citrobacter werkmanii  65
8                 Citrobacter youngae  40
9          Citrobacter_A amalonaticus  57
10       Citrobacter_A amalonaticus_C   1
11       Citrobacter_A amalonaticus_E   2
12               Citrobacter_B koseri 108
13       Citrobacter_C amalonaticus_A   4
14             Cronobacter condimenti   3
15            Cronobacter dublinensis  42
16            Cronobacter malonaticus  62
17             Cronobacter muytjensii   8
18              Cronobacter sakazakii 436
19            Cronobacter universalis   4
20              Enterobacter asburiae  32
21            Enterobacter asburiae_A   6
22            Enterobacter asburiae_B  68
23            Enterobacter asburia

In addition: Warning message:
Expected 8 pieces. Missing pieces filled with `NA` in 1 rows [4957]. 


In [17]:
%%R -o speciesAssemblies

speciesAssemblies <- samples %>%
separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
    mutate(f7 = gsub('_.','',f7)) %>%
    mutate(f7 = gsub('; s','',f7)) %>%
    mutate(f8 = gsub('_.','',f8)) %>%
#	filter(f4 == "Enterobacteriaceae_gtdb") %>%
#	filter(f5 != "other_families") %>%
	count(f7,f8) %>%
	rename(clade = f7, species = f8, assemblies = n)

In addition: Warning message:
Expected 8 pieces. Missing pieces filled with `NA` in 1 rows [4957]. 


In [18]:
%%R
Enterobacteriaceae_best_hits <- hmm_hits %>%
    separate(gtdb_taxonomy, c('f1','f2','f3','f4','f5','f6','f7','f8'), sep = "__") %>%
    mutate(f7 = gsub('_.','',f7)) %>%
    mutate(f7 = gsub('; s','',f7)) %>%
    mutate(f8 = gsub('_.','',f8)) %>%
    rename(clade = f7, species = f8)
head(Enterobacteriaceae_best_hits$species, 20)

 [1] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
 [4] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
 [7] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
[10] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
[13] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
[16] "Klebsiella aerogenes" "Klebsiella aerogenes" "Klebsiella aerogenes"
[19] "Klebsiella aerogenes" "Klebsiella aerogenes"


In [19]:
%%R
Enterobacteriaceae_best_hits %>%
#    filter(component == "microcins.verified") %>%
    count(species, name = "n_microcins", )


# A tibble: 33 × 2
   species                   n_microcins
   <chr>                           <int>
 1 Citrobacter amalonaticus            4
 2 Citrobacter braakii                 6
 3 Citrobacter koseri                  2
 4 Citrobacter werkmanii               3
 5 Citrobacter youngae                 2
 6 Cronobacter dublinensis             3
 7 Cronobacter malonaticus             2
 8 Cronobacter muytjensii              1
 9 Enterobacter asburiae               6
10 Enterobacter cancerogenus          15
# ℹ 23 more rows
# ℹ Use `print(n = ...)` to see more rows


In [20]:
%%R
species_microcins <- Enterobacteriaceae_best_hits %>%
#	filter(component == "microcins.verified") %>%
	count(species, genome) %>% 
	count(species, name = "n_microcins") %>%
	full_join(speciesAssemblies) %>%
	mutate(percent_microcin = n_microcins / assemblies) %>%
    mutate(percent_microcin = gsub("NA", "0", percent_microcin))

# species_microcins

Joining with `by = join_by(species)`


In [21]:
%%R
species_microcins <- species_microcins %>%
    mutate(species = gsub('Klebsiella planticola','Raoultella planticola',species))%>%
    mutate(species = gsub('Klebsiella ornithinolytica','Raoultella ornithinolytica',species))


In [22]:
%%R
Enterobacteriaciae_species_tree$tip.label <- sub("_", " ", Enterobacteriaciae_species_tree$tip.label)

Enterobacteriaciae_species_tree$tip.label <- ifelse(
  Enterobacteriaciae_species_tree$tip.label %in% species_microcins$species,
  paste0(
    Enterobacteriaciae_species_tree$tip.label,
    " (",
    species_microcins$assemblies[match(Enterobacteriaciae_species_tree$tip.label, species_microcins$species)],
    ")"
  ),
  Enterobacteriaciae_species_tree$tip.label
)

# Enterobacteriaciae_species_tree$tip.label


In [23]:
%%R
species_microcins$species <- gsub("_"," ",species_microcins$species)
par(family  = "Arial")
# plot(Enterobacteriaciae_species_tree)

In [24]:
%%R -w 600
species_tree_plot <- ggtree(Enterobacteriaciae_species_tree) +
	geom_tiplab(as_ylab=FALSE, fontface=3, size=3)
showtext_auto()
# species_tree_plot

In [25]:
%%R
species_microcins$Clade <- factor(species_microcins$clade, levels = c("Escherichia","Klebsiella","Enterobacter","Kosakonia","Cronobacter"))


In [26]:
%%R
species_microcins %>%
	filter(!(species %in% Enterobacteriaciae_species_tree$tip.label))

#add assemblies to species in species_microcins
species_microcins$species <- paste0(species_microcins$species, " (", species_microcins$assemblies, ")")
	
#first word before space from species into clade
species_microcins$Clade <- gsub(" .*","",species_microcins$species)


In [27]:
%%R
species_microcins

# A tibble: 52 × 6
   species                   n_microcins clade assemblies percent_microcin Clade
   <chr>                           <int> <chr>      <int> <chr>            <chr>
 1 Citrobacter amalonaticus…           4 Citr…         64 0.0625           Citr…
 2 Citrobacter braakii (119)           6 Citr…        119 0.0504201680672… Citr…
 3 Citrobacter koseri (108)            2 Citr…        108 0.0185185185185… Citr…
 4 Citrobacter werkmanii (6…           3 Citr…         65 0.0461538461538… Citr…
 5 Citrobacter youngae (40)            2 Citr…         40 0.05             Citr…
 6 Cronobacter dublinensis …           3 Cron…         42 0.0714285714285… Cron…
 7 Cronobacter malonaticus …           2 Cron…         62 0.0322580645161… Cron…
 8 Cronobacter muytjensii (…           1 Cron…          8 0.125            Cron…
 9 Enterobacter asburiae (1…           6 Ente…        107 0.0560747663551… Ente…
10 Enterobacter cancerogenu…          14 Ente…         16 0.875            Ente…
# ℹ 42 mo

In [28]:
%%R
species_microcins_hmmer <- species_microcins %>%
	mutate(percent_microcin = n_microcins / assemblies) %>%
    mutate(Clade = gsub('Citrobacter','Escherichia',Clade)) %>%
    mutate(Clade = gsub('Salmonella','Escherichia',Clade)) %>%
    mutate(Clade = gsub('Raoultella','Klebsiella',Clade)) %>%
    mutate(Clade = gsub('Leclercia','Enterobacter',Clade)) %>%
    mutate(Clade = gsub('Lelliottia','Enterobacter',Clade)) %>%
    mutate(Clade = gsub('Atlantibacter','Cronobacter',Clade)) %>%
    mutate(Clade = gsub('Pluralibacter','Klebsiella',Clade))

species_microcins_hmmer$n_microcins[is.na(species_microcins_hmmer$n_microcins)] <- 0
species_microcins_hmmer$percent_microcin[is.na(species_microcins_hmmer$percent_microcin)] <- 0

#count all assemblies, join all
sum(species_microcins_hmmer$assemblies)

[1] 7981


In [29]:
%%R
species_microcins_hmmer$Clade <- factor(species_microcins_hmmer$Clade, levels = c("Escherichia","Klebsiella","Enterobacter","Kosakonia","Cronobacter"))


In [31]:
%%R -w 500

species_tree_microcin_plot <- facet_plot(
    species_tree_plot +
        xlim(0, 100),
        panel = "percent microcin",
        data = species_microcins_hmmer,
        geom = geom_barh,
        mapping = aes(x = (percent_microcin)*100, fill = Clade),
        stat="identity"
    ) +
    scale_fill_manual(values=c(
        "#7397c7ff", 
        "#8fc960ff", 
        "#dfbf5dff",
        "#cf6e6eff",
        "#a674cbff")) +
    theme_tree2(
        plot.margin = margin(0,0,0,0),
        strip.background = element_blank(),
        strip.text.x = element_blank(),
        #text = element_text(size = 14)
        legend.title = element_text(),
        legend.text = element_text(face = 'italic')
        ) +
    xlab("Percent assemblies with microcins")

species_tree_microcin_plot

# update plot file name and add current date from to file name
filename = paste("plots/cinful_old_search.svg")
ggsave(filename)


ℹ invalid tbl_tree object. Missing column: parent,node.
ℹ invalid tbl_tree object. Missing column: parent,node.
Saving 6.94 x 6.67 in image
